In [14]:
from time import time
import warnings
import numpy as np
import pickle as pkl
from parser_all import parse
from qiskit import Aer
from qiskit.utils.quantum_instance import QuantumInstance
from generate_qubos import solve_classically, arr_to_str
# from qiskit.algorithms.optimizers import COBYLA, SLSQP
from classical_optimizers import NLOPT_Optimizer
from QAOAEx import QAOACustom, convert_from_fourier_point, convert_to_fourier_point
from QAOA_methods import (CustomQAOA,
                         generate_points,
                         get_costs,
                         find_all_ground_states,
                         count_coupling_terms,
                         interp_point,
                         construct_initial_state,
                         n_qbit_mixer)
from QAOAEx import convert_from_fourier_point, convert_to_fourier_point

In [16]:
with open('qubos_{}_car_{}_routes/qubo_{}.pkl'.format(3, 3, 3), 'rb') as f:
    qubo, max_coeff, operator, offset, routes = pkl.load(f)
from qiskit_optimization import QuadraticProgram
qubo = QuadraticProgram()
qubo.from_ising(operator)
x_s, opt_value, classical_result = find_all_ground_states(qubo)

In [17]:
optimizer = NLOPT_Optimizer(method = "LN_COBYLA", result_message=False)

In [18]:
backend = Aer.get_backend("statevector_simulator")
quantum_instance = QuantumInstance(backend = backend)

In [19]:
initial_state = construct_initial_state(no_routes = 3, no_cars = 3)
mixer = n_qbit_mixer(initial_state)

In [21]:
p = 2
fourier_point = [-0.07992929, -0.07786024,  0.45417582, -0.13951623]

In [22]:
qaoa = QAOACustom(optimizer = optimizer, reps = p, initial_state = initial_state, mixer = mixer, initial_point = fourier_point, include_custom= True, quantum_instance = quantum_instance)
# qaoa.set_parameterise_point_for_energy_evaluation(convert_from_fourier_point)
# qaoa_results = qaoa.compute_minimum_eigenvalue(operator= operator)

In [26]:
from qiskit.circuit import Parameter
parameters = [ Parameter('u_{}'.format(i)) for i in range(p) ] + [ Parameter('v_{}'.format(i)) for i in range(p) ]
circ = qaoa.construct_circuit(parameter=parameters, operator = operator)[0]

In [28]:
assigned_circ = circ.assign_parameters( {  parameter: i for (i, parameter) in enumerate(parameters) })

In [30]:
from qiskit import execute

In [31]:
result = execute(assigned_circ, backend=backend).result()

In [32]:
from qiskit.quantum_info import Statevector
sv = result.get_statevector()

In [34]:
probs = qaoa._eigenvector_to_solutions(sv, qubo)
total = sum( [item[1]* item[2] for item in probs] )
total

-210.58647555747623

In [36]:
sv_2 = qaoa.calculate_statevector_at_point(operator = operator, point = [0,1,2,3])
sv_2
probs_2 = qaoa._eigenvector_to_solutions(sv_2, qubo)
total_2 = sum( [item[1] * item[2] for item in probs_2] )
total_2

-210.58647555747623

In [39]:
qaoa.reset_reps(4)

In [46]:
%%time
qaoa.set_parameterise_point_for_energy_evaluation(convert_from_fourier_point)
for set_parameterise_point_for_energy_evaluation_ in range(20):
    e = qaoa.evaluate_energy_at_point(operator = operator, point = [-0.0639487, -0.07643202, 0.0395166, -0.03228677, 0.49944737, 0.07051154, 0.08479134, -0.12322544])

CPU times: user 14 s, sys: 1.27 s, total: 15.3 s
Wall time: 17.2 s


In [258]:
probs_2 = qaoa._eigenvector_to_solutions(state, qubo)
sorted(probs_2, key = lambda x: x[2], reverse= True)

0.9999944403978153

In [12]:
%%time
!for i in {0..10}; do python TQA_init_QAOA.py -N 3 -R 3 -M 4 -S ${i} -F -O LN_BOBYQA; done

--------------------------------------------------
QUBO_0
optimal function value: -140.0
optimal value: [0. 1. 0. 0. 0. 1. 1. 0. 0.]
status: SUCCESS
Other ground state(s) found: '001001010'
Other ground state(s) found: '001100001'
Other ground state(s) found: '001100010'
Other ground state(s) found: '010001001'
Other ground state(s) found: '010001010'
Other ground state(s) found: '010100001'
Other ground state(s) found: '010100010'
--------------------------------------------------
Now solving with TQA_QAOA... Fourier Parametrisation: True
--------------------------------------------------
    p=1
('001100001', (-140.0, 0.038339170884521906))
('001001001', (-140.0, 0.03876357985428722))
('010100001', (-140.0, 0.038858258009332694))
('001100010', (-140.0, 0.038931791056402994))
('010001001', (-140.0, 0.03930725219517446))
    Optimal_point: [0.3844230416503986, 0.0401739087878706]
    Exp_val: -132.58986065417534, Prob_s: 0.0391404356883734, approx_ratio: 0.9470704332441096
------------